In [9]:
import base64
import csv
import pandas as pd
import numpy as np
import pprint
import urllib.request, urllib.parse, urllib.error
import httplib2
from xml.dom import minidom
import getpass

pp = pprint.PrettyPrinter()

fn = "log64_attempts.12-13-2012.csv"

indexes = " OR ".join(["index="+x for x in """directory_services_nonprod
dmv_google_cloud
doh_google_cloud
dol_google_cloud
dtf_http
estreamer
fireeye
hcr_google_cloud
health_datapower
health_network
its_google_cloud
its_okta_nonprod
main
miauditlogs
os
otda_google_cloud
vmware
webny
webny_nonprod
wineventlog""".split("\n")])


def d64(x):
    # d(b64) decodes base64 byte string. =Nan if there's an error
    try:
        r = base64.b64decode(x, altchars="-_").decode()
    except:
        r = np.nan
    return r

def make_list(events):
    # 
    lst = []
    for result in results.ResultsReader(events):
        lst.extend(result.values())
    return lst

def list_to_df(lst, columns):
    '''Create a df with n=len(columns) columns from alist
       0th row has items lst[0] lst[1] ... (n-1)th
       lst row has items lst[0+n] lst[1+n] ... lst[(n-1 + n)i
       ... etc
    '''
    num_columns = len(columns)
    len_list    = len(lst)
    
    # Make sure t
    if len_list % num_columns != 0:
        print(f"length of lst ({len_list}) must be even multiple of # of names ({num_columns})")
        return None
    
    # Reshape the list into a list of lists before returning the dataframe
    sublists = [ lst[i:i+num_columns] for i in range(0, len_list, num_columns) ]
    return pd.DataFrame(sublists, columns=columns)

# This job makes a real search and outputs the result.
# Assume an authenticated session is open.

import sys
from time import sleep
import splunklib.results as results
import splunklib.client as client

HOST = "cnsesplunkoperations.svc.ny.gov"
PORT = 8089
USERNAME = "bdk01"
PASSWORD = getpass.getpass()
OWNER = "bdk01"
APP = "search"

# Create a Service instance and log in 
service = client.connect(
    host=HOST,
    port=PORT,
    username=USERNAME,
    password=PASSWORD,
    owner=OWNER,
    app=APP)

# Build a query
# earliest_time = "12/13/2021:00:00:00"
earliest_time = "-2d@d"
latest_time ="-1d@d"
detail = '''Base64 Command "${jndi:*}" | rex "\${jndi.*?Base64\/(?<b64>[^}]*?)}.*" 
| mvexpand b64 | stats values(host) as host earliest(_time) as earliest  by b64 |  mvexpand host 
| convert timeformat="%m/%d/%Y %H:%M:%S" ctime(earliest) as earliest 
| sort host +earliest 
| table host earliest b64'''

query = " ".join(["search earliest=", earliest_time, "latest=", latest_time, indexes, detail])

# searchquery_normal = "search index=main | stats values(sourcetype)"
searchquery_normal = query
# pp.pprint(query)

kwargs_normalsearch = {
#     "earliest_time": "12/09/2021:00:00:00",
#     "latest_time": "-1d@d",
    "search_mode": "normal"
    }

job = service.jobs.create(searchquery_normal, **kwargs_normalsearch)

# A normal search returns the job's SID right away, so we need to poll for completion
while True:
    while not job.is_ready():
        pass
    stats = {"isDone": job["isDone"],
             "doneProgress": float(job["doneProgress"])*100,
              "scanCount": int(job["scanCount"]),
              "eventCount": int(job["eventCount"]),
              "resultCount": int(job["resultCount"])}

    status = ("\r%(doneProgress)03.1f%%   %(scanCount)d scanned   "
              "%(eventCount)d matched   %(resultCount)d results") % stats

    print(f"{status=}")
    sys.stdout.flush()
    if stats["isDone"] == "1":
        sys.stdout.write("\n\nDone!\n\n")
        break
    sleep(2)

# Get the results and display them
# for result in results.ResultsReader(job.results()):
#     print(result)

save_results = job.results()

lst = make_list(save_results)
log = list_to_df(lst, columns=["host", "earliest", "base64_payload"])

# Decode the b64 column into new column, payload
log["payload"] = log.loc[:,"base64_payload"].apply(d64)

# Show the result or save the result 
# pp.pprint(log[["host", "earliest", "payload"]])

log[["host", "earliest", "payload"]].to_csv("search_log4j.txt", index=False)

job.cancel()   
sys.stdout.write('\n')



········
status='\r0.0%   0 scanned   0 matched   0 results'
status='\r100.0%   1844 scanned   222 matched   140 results'


Done!




1

In [8]:
''' Read report of web payload's and get list of unique source hosts '''

import re
import pandas as pd
import numpy as np

def fetch_ip(cmd):
    grp = re.findall(ippat, cmd)
    grp = [] if not grp else grp
    for i,x in enumerate(grp):
        grp[i] = ".".join(x)
#     import pdb; pdb.set_trace()
    return grp

def fetch_host(cmd):
    grp = re.findall(httppat, cmd)
#     import pdb; pdb.set_trace()
    return [] if not grp else grp

# read the search results
fn = "search_log4j.txt"
log = pd.read_csv(fn)

# Clean up
# Drop any rows that contain Nan. e.g. some of the web paylod's 
# cannot be converted from base64
log = log.dropna(subset=["payload"]).reset_index(drop=True)

## Find the hosts being referenced in the payload's

ippat = r"([0-9]{1,3})\.([0-9]{1,3})\.([0-9]{1,3})\.([0-9]{1,3})"
httppat = r"https?:\/\/([^\/:]+)"

hosts = {}

grp = []
for cmd in log.payload:
    r1 = fetch_ip(cmd)
    r2 = fetch_host(cmd)
    grp.extend(r1)
    grp.extend(r2)
        
hosts = set(grp)

import pprint
pp = pprint.PrettyPrinter()

# str = "(" + " OR ".join([f"SourceIp={x}" for x in hosts]) + ")"
str = "(" + " OR ".join([f"{x}" for x in hosts]) + ")"


# pp.pprint(hosts)
str


        



'(170.123.239.220 OR 35.241.56.154 OR 170.123.239.66 OR 34.117.162.9 OR 34.117.27.241 OR 170.123.239.86 OR 54.37.93.31 OR 134.179.112.155 OR 34.149.252.48 OR 177.52.40.22 OR 195.54.160.149 OR 170.123.239.207 OR 141.105.65.94 OR 35.227.230.102 OR 170.123.239.133 OR 170.123.233.59 OR 34.117.14.72 OR 34.102.236.102 OR 170.123.239.132 OR 35.190.73.201 OR 34.117.243.152 OR 155.94.154.170 OR 34.107.190.113 OR 12.152.252.179 OR 34.102.182.99 OR 35.244.239.250 OR 170.123.233.23 OR 161.11.143.6 OR 34.117.150.208 OR 170.123.239.8 OR www.econ-jobs.com OR 170.123.233.62 OR 170.123.239.9 OR 170.123.239.221 OR 34.102.190.142 OR 34.117.110.198 OR 170.123.234.189 OR 34.120.255.218 OR 170.123.237.72 OR 170.123.239.56 OR 161.11.143.10 OR 34.117.41.213 OR 134.179.112.191 OR 170.123.239.156 OR 34.117.19.113 OR 170.123.233.61 OR 34.117.8.187 OR 170.123.239.136 OR 170.123.239.176 OR 34.96.91.6 OR 170.123.239.47 OR 170.123.239.202 OR 134.179.112.181 OR 170.123.239.139 OR 3.85.59.114 OR 34.107.140.86 OR 35.20

In [ ]:
hos

In [ ]:
str
